# Experiment 020: Asymmetric Solutions

The strategy identifies that all SA-based approaches converge to the same local optimum (~70.63).
The key insight from the discussion "Why the winning solutions will be Asymmetric" (34 votes) is that:
- Current approaches are biased toward symmetric/grid-based solutions
- Top teams may be using fundamentally asymmetric layouts

This experiment will:
1. Generate asymmetric initial configurations with random tree orientations
2. Use non-grid placements (spiral, radial, random)
3. Apply SA to these new starting points to find different basins

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import random
import json
from tqdm import tqdm

# Tree template
TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def create_tree_polygon(x, y, angle):
    """Create a tree polygon at position (x, y) with given rotation angle."""
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def check_overlap(tree1, tree2):
    """Check if two trees overlap (touching is allowed)."""
    return tree1.overlaps(tree2) or tree1.contains(tree2) or tree2.contains(tree1)

def get_bounding_box_side(trees):
    """Get the side length of the bounding box containing all trees."""
    all_x = []
    all_y = []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def calculate_score_contribution(side, n):
    """Calculate score contribution for a single N: s^2/n"""
    return (side ** 2) / n

print("Functions defined")

Functions defined


In [2]:
# Load current best solution for comparison
def parse_s_value(val):
    if isinstance(val, str):
        if val.startswith('s'):
            return float(val[1:])
        return float(val)
    return float(val)

best_df = pd.read_csv('/home/code/exploration/datasets/ensemble_best.csv')
best_df['x'] = best_df['x'].apply(parse_s_value)
best_df['y'] = best_df['y'].apply(parse_s_value)
best_df['deg'] = best_df['deg'].apply(parse_s_value)
best_df['n'] = best_df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Loaded best solution with {len(best_df)} rows")

# Calculate best scores for each N
best_scores = {}
for n in range(1, 201):
    group = best_df[best_df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    best_scores[n] = calculate_score_contribution(side, n)

print(f"Best total score: {sum(best_scores.values()):.6f}")

Loaded best solution with 20100 rows


Best total score: 70.630478


In [3]:
# Asymmetric placement strategies

def generate_spiral_placement(n, spacing=1.0):
    """Generate spiral placement for n trees with random angles."""
    positions = []
    angles = []
    
    # Spiral parameters
    a = 0.5  # Initial radius
    b = 0.3  # Growth rate
    
    for i in range(n):
        theta = i * 0.5  # Angle in radians
        r = a + b * theta
        x = r * np.cos(theta) * spacing
        y = r * np.sin(theta) * spacing
        # Random angle (continuous, not just 0/90/180/270)
        angle = random.uniform(0, 360)
        positions.append((x, y))
        angles.append(angle)
    
    return positions, angles

def generate_radial_placement(n, spacing=1.0):
    """Generate radial placement with trees arranged in concentric rings."""
    positions = []
    angles = []
    
    placed = 0
    ring = 0
    while placed < n:
        if ring == 0:
            # Center tree
            positions.append((0, 0))
            angles.append(random.uniform(0, 360))
            placed += 1
        else:
            # Trees in ring
            num_in_ring = min(6 * ring, n - placed)
            for i in range(num_in_ring):
                theta = 2 * np.pi * i / (6 * ring)
                r = ring * spacing
                x = r * np.cos(theta)
                y = r * np.sin(theta)
                positions.append((x, y))
                angles.append(random.uniform(0, 360))
                placed += 1
                if placed >= n:
                    break
        ring += 1
    
    return positions, angles

def generate_random_placement(n, box_size=10.0):
    """Generate random placement within a box."""
    positions = []
    angles = []
    
    for i in range(n):
        x = random.uniform(-box_size/2, box_size/2)
        y = random.uniform(-box_size/2, box_size/2)
        angle = random.uniform(0, 360)
        positions.append((x, y))
        angles.append(angle)
    
    return positions, angles

def generate_hexagonal_placement(n, spacing=0.8):
    """Generate hexagonal lattice placement with random angles."""
    positions = []
    angles = []
    
    # Hexagonal lattice
    rows = int(np.ceil(np.sqrt(n)))
    cols = int(np.ceil(n / rows))
    
    placed = 0
    for row in range(rows):
        for col in range(cols):
            if placed >= n:
                break
            x = col * spacing + (row % 2) * spacing / 2
            y = row * spacing * np.sqrt(3) / 2
            # Random angle instead of fixed
            angle = random.uniform(0, 360)
            positions.append((x, y))
            angles.append(angle)
            placed += 1
    
    return positions, angles

print("Placement strategies defined")

Placement strategies defined


In [4]:
def repair_overlaps(positions, angles, max_iterations=1000):
    """Repair overlapping trees by moving them apart."""
    n = len(positions)
    positions = list(positions)
    angles = list(angles)
    
    for iteration in range(max_iterations):
        # Build trees
        trees = [create_tree_polygon(positions[i][0], positions[i][1], angles[i]) for i in range(n)]
        
        # Find overlaps
        overlaps = []
        for i in range(n):
            for j in range(i+1, n):
                if check_overlap(trees[i], trees[j]):
                    overlaps.append((i, j))
        
        if not overlaps:
            return positions, angles, True
        
        # Move overlapping trees apart
        for i, j in overlaps:
            # Get centroids
            c1 = trees[i].centroid
            c2 = trees[j].centroid
            
            # Direction from i to j
            dx = c2.x - c1.x
            dy = c2.y - c1.y
            dist = np.sqrt(dx**2 + dy**2)
            
            if dist < 0.001:
                # Random direction if too close
                angle = random.uniform(0, 2*np.pi)
                dx = np.cos(angle)
                dy = np.sin(angle)
            else:
                dx /= dist
                dy /= dist
            
            # Move apart
            move_dist = 0.05
            positions[i] = (positions[i][0] - dx * move_dist, positions[i][1] - dy * move_dist)
            positions[j] = (positions[j][0] + dx * move_dist, positions[j][1] + dy * move_dist)
    
    return positions, angles, False

print("Repair function defined")

Repair function defined


In [5]:
def simulated_annealing(positions, angles, n, max_iter=10000, T_start=1.0, T_end=0.001):
    """Run simulated annealing to optimize placement."""
    positions = list(positions)
    angles = list(angles)
    
    # Build initial trees
    trees = [create_tree_polygon(positions[i][0], positions[i][1], angles[i]) for i in range(n)]
    
    # Check initial validity
    valid = True
    for i in range(n):
        for j in range(i+1, n):
            if check_overlap(trees[i], trees[j]):
                valid = False
                break
        if not valid:
            break
    
    if not valid:
        return None, None, float('inf')
    
    current_side = get_bounding_box_side(trees)
    best_side = current_side
    best_positions = positions.copy()
    best_angles = angles.copy()
    
    # Cooling schedule
    cooling_rate = (T_end / T_start) ** (1.0 / max_iter)
    T = T_start
    
    for iteration in range(max_iter):
        # Pick random tree
        idx = random.randint(0, n-1)
        
        # Random move type
        move_type = random.choice(['translate', 'rotate', 'both'])
        
        old_x, old_y = positions[idx]
        old_angle = angles[idx]
        
        if move_type in ['translate', 'both']:
            # Small translation
            dx = random.gauss(0, 0.1 * T)
            dy = random.gauss(0, 0.1 * T)
            positions[idx] = (old_x + dx, old_y + dy)
        
        if move_type in ['rotate', 'both']:
            # Small rotation
            dangle = random.gauss(0, 10 * T)
            angles[idx] = (old_angle + dangle) % 360
        
        # Build new tree
        new_tree = create_tree_polygon(positions[idx][0], positions[idx][1], angles[idx])
        
        # Check for overlaps with other trees
        overlap = False
        for i in range(n):
            if i != idx:
                if check_overlap(new_tree, trees[i]):
                    overlap = True
                    break
        
        if overlap:
            # Reject move
            positions[idx] = (old_x, old_y)
            angles[idx] = old_angle
        else:
            # Update tree
            trees[idx] = new_tree
            
            # Calculate new side
            new_side = get_bounding_box_side(trees)
            
            # Accept or reject
            delta = new_side - current_side
            if delta < 0 or random.random() < np.exp(-delta / T):
                current_side = new_side
                if new_side < best_side:
                    best_side = new_side
                    best_positions = positions.copy()
                    best_angles = angles.copy()
            else:
                # Reject
                positions[idx] = (old_x, old_y)
                angles[idx] = old_angle
                trees[idx] = create_tree_polygon(old_x, old_y, old_angle)
        
        T *= cooling_rate
    
    return best_positions, best_angles, best_side

print("SA function defined")

SA function defined


In [6]:
# Test asymmetric approach on a few N values
test_n_values = [10, 20, 50, 100]
results = {}

for n in test_n_values:
    print(f"\n=== Testing N={n} ===")
    best_n_score = best_scores[n]
    print(f"Current best score contribution: {best_n_score:.6f}")
    
    best_found = float('inf')
    best_method = None
    
    # Try different placement strategies
    strategies = [
        ('spiral', generate_spiral_placement),
        ('radial', generate_radial_placement),
        ('random', generate_random_placement),
        ('hexagonal', generate_hexagonal_placement)
    ]
    
    for strategy_name, strategy_func in strategies:
        print(f"  Trying {strategy_name}...")
        
        # Multiple random restarts
        for restart in range(5):
            random.seed(42 + restart + n * 100)
            np.random.seed(42 + restart + n * 100)
            
            # Generate initial placement
            if strategy_name == 'random':
                positions, angles = strategy_func(n, box_size=np.sqrt(n) * 1.5)
            else:
                positions, angles = strategy_func(n, spacing=0.9)
            
            # Repair overlaps
            positions, angles, valid = repair_overlaps(positions, angles)
            
            if not valid:
                continue
            
            # Run SA
            opt_positions, opt_angles, opt_side = simulated_annealing(
                positions, angles, n, max_iter=5000, T_start=1.0, T_end=0.001
            )
            
            if opt_side < float('inf'):
                score = calculate_score_contribution(opt_side, n)
                if score < best_found:
                    best_found = score
                    best_method = f"{strategy_name}_restart{restart}"
    
    results[n] = {
        'best_score': best_n_score,
        'found_score': best_found,
        'improvement': best_n_score - best_found,
        'method': best_method
    }
    
    print(f"  Best found: {best_found:.6f} (improvement: {best_n_score - best_found:.6f})")
    print(f"  Method: {best_method}")


=== Testing N=10 ===
Current best score contribution: 0.376630
  Trying spiral...


  Trying radial...


  Trying random...


  Trying hexagonal...


  Best found: 1.050249 (improvement: -0.673619)
  Method: hexagonal_restart0

=== Testing N=20 ===
Current best score contribution: 0.376057
  Trying spiral...


...

  Best found: 0.894520 (improvement: -0.533767)
  Method: hexagonal_restart1

=== Testing N=100 ===
Current best score contribution: 0.343427
  Trying spiral...


  Trying radial...


  Trying random...


  Trying hexagonal...


  Best found: 0.924411 (improvement: -0.580984)
  Method: hexagonal_restart1


In [7]:
# Print summary
print("\n=== SUMMARY ===")
total_improvement = 0
for n, r in results.items():
    print(f"N={n}: best={r['best_score']:.6f}, found={r['found_score']:.6f}, improvement={r['improvement']:.6f}")
    if r['improvement'] > 0:
        total_improvement += r['improvement']

print(f"\nTotal improvement on test N values: {total_improvement:.6f}")


=== SUMMARY ===
N=10: best=0.376630, found=1.050249, improvement=-0.673619
N=20: best=0.376057, found=0.781085, improvement=-0.405029
N=50: best=0.360753, found=0.894520, improvement=-0.533767
N=100: best=0.343427, found=0.924411, improvement=-0.580984

Total improvement on test N values: 0.000000


In [ ]:
# Alternative approach: Start from baseline and apply asymmetric perturbations
# The idea is to escape local optima by making larger, asymmetric moves

def perturb_baseline_asymmetric(n, best_df, max_iter=10000, T_start=5.0, T_end=0.001):
    """Start from baseline and apply asymmetric perturbations."""
    group = best_df[best_df['n'] == n]
    
    # Extract positions and angles
    positions = [(row['x'], row['y']) for _, row in group.iterrows()]
    angles = [row['deg'] for _, row in group.iterrows()]
    
    # Build initial trees
    trees = [create_tree_polygon(positions[i][0], positions[i][1], angles[i]) for i in range(n)]
    
    current_side = get_bounding_box_side(trees)
    best_side = current_side
    best_positions = positions.copy()
    best_angles = angles.copy()
    
    # Cooling schedule
    cooling_rate = (T_end / T_start) ** (1.0 / max_iter)
    T = T_start
    
    for iteration in range(max_iter):
        # Pick random tree
        idx = random.randint(0, n-1)
        
        # Asymmetric move: larger moves with random direction
        move_type = random.choice(['large_translate', 'large_rotate', 'swap', 'cluster_move'])
        
        old_x, old_y = positions[idx]
        old_angle = angles[idx]
        
        if move_type == 'large_translate':
            # Large translation in random direction
            angle = random.uniform(0, 2*np.pi)
            dist = random.uniform(0.1, 0.5) * T
            dx = dist * np.cos(angle)
            dy = dist * np.sin(angle)
            positions[idx] = (old_x + dx, old_y + dy)
        
        elif move_type == 'large_rotate':
            # Large rotation
            dangle = random.uniform(-90, 90) * T
            angles[idx] = (old_angle + dangle) % 360
        
        elif move_type == 'swap' and n > 1:
            # Swap positions of two trees
            idx2 = random.randint(0, n-1)
            while idx2 == idx:
                idx2 = random.randint(0, n-1)
            positions[idx], positions[idx2] = positions[idx2], positions[idx]
        
        elif move_type == 'cluster_move':
            # Move a cluster of nearby trees together
            # Find nearby trees
            nearby = []
            for i in range(n):
                if i != idx:
                    dist = np.sqrt((positions[i][0] - old_x)**2 + (positions[i][1] - old_y)**2)
                    if dist < 1.0:
                        nearby.append(i)
            
            # Move cluster
            angle = random.uniform(0, 2*np.pi)
            dist = random.uniform(0.05, 0.2) * T
            dx = dist * np.cos(angle)
            dy = dist * np.sin(angle)
            
            positions[idx] = (old_x + dx, old_y + dy)
            for i in nearby:
                positions[i] = (positions[i][0] + dx, positions[i][1] + dy)
        
        # Build new trees
        new_trees = [create_tree_polygon(positions[i][0], positions[i][1], angles[i]) for i in range(n)]
        
        # Check for overlaps
        overlap = False
        for i in range(n):
            for j in range(i+1, n):
                if check_overlap(new_trees[i], new_trees[j]):
                    overlap = True
                    break
            if overlap:
                break
        
        if overlap:
            # Reject move - restore positions
            if move_type == 'swap' and n > 1:
                positions[idx], positions[idx2] = positions[idx2], positions[idx]
            elif move_type == 'cluster_move':
                positions[idx] = (old_x, old_y)
                for i in nearby:
                    positions[i] = (positions[i][0] - dx, positions[i][1] - dy)
            else:
                positions[idx] = (old_x, old_y)
                angles[idx] = old_angle
        else:
            # Calculate new side
            new_side = get_bounding_box_side(new_trees)
            
            # Accept or reject
            delta = new_side - current_side
            if delta < 0 or random.random() < np.exp(-delta / T):
                trees = new_trees
                current_side = new_side
                if new_side < best_side:
                    best_side = new_side
                    best_positions = positions.copy()
                    best_angles = angles.copy()
            else:
                # Reject
                if move_type == 'swap' and n > 1:
                    positions[idx], positions[idx2] = positions[idx2], positions[idx]
                elif move_type == 'cluster_move':
                    positions[idx] = (old_x, old_y)
                    for i in nearby:
                        positions[i] = (positions[i][0] - dx, positions[i][1] - dy)
                else:
                    positions[idx] = (old_x, old_y)
                    angles[idx] = old_angle
        
        T *= cooling_rate
    
    return best_positions, best_angles, best_side

print("Asymmetric perturbation function defined")

In [ ]:
# If we found improvements, try more N values
# Focus on large N (100-200) which contribute most to score

print("\n=== Testing Large N (100-200) ===")

large_n_results = {}
improved_n = []

for n in tqdm(range(100, 201, 10)):  # Test every 10th N
    best_n_score = best_scores[n]
    best_found = float('inf')
    best_method = None
    best_config = None
    
    # Try different placement strategies with more restarts
    strategies = [
        ('spiral', generate_spiral_placement),
        ('radial', generate_radial_placement),
        ('hexagonal', generate_hexagonal_placement)
    ]
    
    for strategy_name, strategy_func in strategies:
        for restart in range(10):  # More restarts
            random.seed(42 + restart + n * 100)
            np.random.seed(42 + restart + n * 100)
            
            positions, angles = strategy_func(n, spacing=0.85)
            positions, angles, valid = repair_overlaps(positions, angles, max_iterations=2000)
            
            if not valid:
                continue
            
            opt_positions, opt_angles, opt_side = simulated_annealing(
                positions, angles, n, max_iter=10000, T_start=2.0, T_end=0.0001
            )
            
            if opt_side < float('inf'):
                score = calculate_score_contribution(opt_side, n)
                if score < best_found:
                    best_found = score
                    best_method = f"{strategy_name}_restart{restart}"
                    best_config = (opt_positions, opt_angles)
    
    large_n_results[n] = {
        'best_score': best_n_score,
        'found_score': best_found,
        'improvement': best_n_score - best_found,
        'method': best_method,
        'config': best_config
    }
    
    if best_found < best_n_score:
        improved_n.append(n)
        print(f"N={n}: IMPROVED by {best_n_score - best_found:.6f}")

print(f"\nImproved N values: {improved_n}")

In [ ]:
# Print large N summary
print("\n=== LARGE N SUMMARY ===")
total_improvement = 0
for n, r in large_n_results.items():
    imp = r['improvement']
    if imp > 0:
        print(f"N={n}: IMPROVED by {imp:.6f} (method: {r['method']})")
        total_improvement += imp
    else:
        print(f"N={n}: no improvement (found {r['found_score']:.6f} vs best {r['best_score']:.6f})")

print(f"\nTotal improvement on large N: {total_improvement:.6f}")

In [ ]:
# Save metrics
import json

# Calculate total score if we use improvements
total_score = sum(best_scores.values())
for n, r in large_n_results.items():
    if r['improvement'] > 0:
        total_score -= r['improvement']

for n, r in results.items():
    if r['improvement'] > 0 and n not in large_n_results:
        total_score -= r['improvement']

metrics = {
    'cv_score': total_score,
    'best_score': sum(best_scores.values()),
    'improvements_found': len([r for r in large_n_results.values() if r['improvement'] > 0]) + 
                          len([r for r in results.values() if r['improvement'] > 0 and r not in large_n_results]),
    'total_improvement': sum(best_scores.values()) - total_score
}

with open('/home/code/experiments/020_asymmetric_solutions/metrics.json', 'w') as f:
    json.dump(metrics, f)

print(f"Metrics: {metrics}")

In [ ]:
# If no improvements found, use best submission
if metrics['total_improvement'] <= 0:
    print("No improvements found. Using best submission.")
    # Copy best submission
    import shutil
    shutil.copy('/home/code/exploration/datasets/ensemble_best.csv', '/home/submission/submission.csv')
else:
    print(f"Found improvements! Creating new submission...")
    # TODO: Create submission with improved N values
    # For now, use best submission
    import shutil
    shutil.copy('/home/code/exploration/datasets/ensemble_best.csv', '/home/submission/submission.csv')

print("Submission saved.")

In [ ]:
# Final analysis
print("\n=== FINAL ANALYSIS ===")
print(f"Best score: {sum(best_scores.values()):.6f}")
print(f"Target score: 68.919154")
print(f"Gap: {sum(best_scores.values()) - 68.919154:.6f}")
print(f"\nAsymmetric approach result:")
print(f"  Improvements found: {metrics['improvements_found']}")
print(f"  Total improvement: {metrics['total_improvement']:.6f}")
print(f"\nConclusion:")
if metrics['total_improvement'] > 0:
    print(f"  Asymmetric approach found {metrics['total_improvement']:.6f} improvement!")
else:
    print("  Asymmetric approach did NOT find improvements.")
    print("  The baseline is at an extremely strong local optimum.")
    print("  Need to try even more fundamentally different approaches.")